## Import

In [63]:
# Data Librairies
import pandas as pd
import numpy as np

# Data Modeling
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score
import xgboost as xgb

# Data Vizualization Librairies
import seaborn as sns

# Warnings
import warnings
warnings.filterwarnings("ignore")

## Fonctions utilitaires

In [43]:
def getAgeCat(age: int) -> str:
    cat: str = ""

    if(age < 35):
        cat = "34 & -"
    elif (age >=35 and age < 45):
        cat = "35-44"
    elif (age >= 45 and age < 55):
        cat = "45-54"
    elif (age >= 55 and age < 65):
        cat = "55-64"
    else:
        cat = "65 & +"

    return cat

### Load Dataset

In [44]:
df = pd.read_csv("./Data/data_participant.csv", index_col=None)
df.describe()

,ID,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default-payment-next-month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,24000.000000
mean,15000.500000,167484.322667,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100,51223.330900,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.252333
std,8660.398374,129747.661567,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988,73635.860576,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.434361
min,1.000000,10000.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3558.750000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22381.500000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67091.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,1.000000
max,30000.000000,1000000.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [45]:
# Create "Remaining Amount" Column
df["RMN_AMT2"] = df["BILL_AMT2"] - df["PAY_AMT1"]
df["RMN_AMT3"] = df["BILL_AMT3"] - df["PAY_AMT2"]
df["RMN_AMT4"] = df["BILL_AMT4"] - df["PAY_AMT3"]
df["RMN_AMT5"] = df["BILL_AMT5"] - df["PAY_AMT4"]
df["RMN_AMT6"] = df["BILL_AMT6"] - df["PAY_AMT5"]

# Create "Sufficient Payment"
df["SUFFICIENT_PAYMENT_AMT2"] = np.where(df["RMN_AMT2"] <= 0, 1, 0)
df["SUFFICIENT_PAYMENT_AMT3"] = np.where(df["RMN_AMT3"] <= 0, 1, 0)
df["SUFFICIENT_PAYMENT_AMT4"] = np.where(df["RMN_AMT4"] <= 0, 1, 0)
df["SUFFICIENT_PAYMENT_AMT5"] = np.where(df["RMN_AMT5"] <= 0, 1, 0)
df["SUFFICIENT_PAYMENT_AMT6"] = np.where(df["RMN_AMT6"] <= 0, 1, 0)

# Add weight to most recent months
df["SUFFICIENT_PAYMENT_AMT2"] = df["SUFFICIENT_PAYMENT_AMT2"]*1.4
df["SUFFICIENT_PAYMENT_AMT3"] = df["SUFFICIENT_PAYMENT_AMT3"]*1.3
df["SUFFICIENT_PAYMENT_AMT4"] = df["SUFFICIENT_PAYMENT_AMT4"]*1.2
df["SUFFICIENT_PAYMENT_AMT5"] = df["SUFFICIENT_PAYMENT_AMT5"]*1.1
df["SUFFICIENT_PAYMENT_AMT6"] = df["SUFFICIENT_PAYMENT_AMT6"]*1

df.drop(['RMN_AMT2', 'RMN_AMT3', 'RMN_AMT4', 'RMN_AMT5', 'RMN_AMT6'], axis=1)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default-payment-next-month,SUFFICIENT_PAYMENT_AMT2,SUFFICIENT_PAYMENT_AMT3,SUFFICIENT_PAYMENT_AMT4,SUFFICIENT_PAYMENT_AMT5,SUFFICIENT_PAYMENT_AMT6
0,15065,240000,male,graduate school,married,46,-2,-2,-2,-2,...,11705,9473,1405,164705,1.0,1.4,1.3,1.2,1.1,1
1,29264,80000,male,high school,single,23,-1,2,-1,-1,...,125,0,0,0,0.0,0.0,1.3,1.2,1.1,1
2,26266,30000,female,university,married,46,1,2,0,0,...,1200,500,737,0,0.0,0.0,0.0,0.0,0.0,0
3,2371,210000,male,university,single,28,0,0,0,0,...,6500,6500,6300,7900,0.0,0.0,0.0,0.0,0.0,0
4,20112,230000,female,graduate school,married,36,-2,-2,-2,-2,...,2427,282,282,1886,0.0,1.4,1.3,1.2,1.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,12269,110000,female,university,single,25,2,0,0,0,...,3000,5000,10000,3000,NaN,0.0,0.0,0.0,0.0,0
29996,22101,50000,female,university,single,28,0,0,0,0,...,1446,1500,1500,1073,NaN,0.0,0.0,0.0,0.0,0
29997,1380,80000,male,high school,single,26,1,2,2,2,...,2800,0,7000,8,NaN,0.0,0.0,0.0,0.0,0
29998,8396,360000,female,graduate school,single,29,0,0,0,0,...,41,22,9865,14847,NaN,0.0,0.0,0.0,0.0,1


In [46]:
df.describe(exclude=np.number) # Describe Categorical Values

,SEX,EDUCATION,MARRIAGE
count,30000,30000,30000
unique,2,5,4
top,female,university,single
freq,18112,14030,15964


In [47]:
cats = df.select_dtypes(exclude=np.number).columns.tolist()
for cat in cats:
    df[cat] = df[cat].astype('category')

In [48]:
# Finding Protected Groups
df['group'] = df.apply(lambda x: x['SEX'] + '-' + x['EDUCATION'] + '-' + x['MARRIAGE'], axis=1)
noDefault = df[df['default-payment-next-month'] == 0].groupby(['group'])['default-payment-next-month'].count().rename("noDefault")
default = df[df['default-payment-next-month'] == 1].groupby(['group'])['default-payment-next-month'].count().rename("default")
df_groups = pd.concat([noDefault, default], axis=1).fillna(0)
df_groups["total"] = df_groups.apply(lambda x : x["noDefault"] + x["default"], axis=1)
df_groups["propDefault"] = df_groups.apply(lambda x : x["default"]/x["total"], axis=1)
propDefault_mean = df_groups["propDefault"].mean()
propDefault_median = df_groups["propDefault"].median()

df_groups['protected'] = np.where(df_groups['propDefault'] > propDefault_median, 1, 0)
df_groups

,noDefault,default,total,propDefault,protected
group,,,,,
female-graduate school-married,1259,359.0,1618.0,0.221879,1
female-graduate school-others,14,1.0,15.0,0.066667,0
female-graduate school-single,2773,531.0,3304.0,0.160714,0
female-graduate school-unknown,3,0.0,3.0,0.000000,0
female-high school-married,981,481.0,1462.0,0.329001,1
female-high school-others,37,17.0,54.0,0.314815,1
female-high school-single,600,199.0,799.0,0.249061,1
female-high school-unknown,23,5.0,28.0,0.178571,0
female-others-married,26,2.0,28.0,0.071429,0


In [49]:
# Tagging Protected Group
df = df.merge(df_groups['protected'], on='group')
df.set_index(['ID'], inplace=True)
df.drop(['group'], axis=1, inplace=True)

In [50]:
# Creating Target and Explicative Datasets
df_training = df[(~df['default-payment-next-month'].isna())]
df_prediction = df[(df['default-payment-next-month'].isna())]

X, y = df_training.drop(['default-payment-next-month'], axis=1), df_training['default-payment-next-month']

## Creating Train and Test Datasets

In [51]:
# Standard Sklearn Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)
# XGBoost Matrix
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [60]:
# Defining Hyperparameters
params = {
    "objective": "binary:logistic",
    # "num_class": 2
    # "tree_method": "gpu_hist"
}

In [61]:
# Defining and Training Model
n = 50
model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
)

In [62]:
# RMSE Validation
preds = model.predict(dtest_reg)
rmse = mean_squared_error(y_test, preds, squared=False)

preds_xgb = np.where(preds < 0.5, 0, 1)
print(accuracy_score(y_test, preds_xgb))
print(f1_score(y_test, preds_xgb))

print(f"RMSE of the base model: {rmse:.3f}")

0.781
0.42720139494333037
RMSE of the base model: 0.402


In [55]:
# F1_Score Validation

In [56]:
# Faireness Validation